In [1]:
!pip install ultralytics pillow tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 kB 20.9 MB/s eta 0:00:00


In [2]:
!pip install ultralytics pillow tqdm
from ultralytics import YOLO # update the import statement for YOLO
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from ultralytics.utils.downloads import download # Import the download function

def visdrone2yolo(dir):
    """
    Converts VisDrone dataset annotations to YOLO format.

    Args:
        dir (Path): Path to the VisDrone dataset directory (e.g., 'VisDrone2019-DET-train').
    """

    def convert_box(size, box):
        """Converts a VisDrone bounding box to YOLO xywh format."""
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    # Create 'labels' directory if it doesn't exist
    (dir / 'labels').mkdir(parents=True, exist_ok=True)

    # Loop through annotation files and convert them
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
    for f in pbar:
        img_size = Image.open((dir / 'images' / f.name).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # Skip 'ignored regions'
                    continue
                cls = int(row[5]) - 1  # Get class label (adjusting for 0-based indexing)
                box = convert_box(img_size, tuple(map(int, row[:4])))  # Convert box format
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
        with open(str(f).replace(os.sep + 'annotations' + os.sep, os.sep + 'labels' + os.sep), 'w') as fl:
            fl.writelines(lines)  # Write converted annotations to 'labels' directory

# --- Main execution ---
# Assume 'yaml' is a dictionary containing dataset path information
# make sure to replace 'your_dataset_path'
yaml = {'path': './dataset'}

dir = Path(yaml['path'])  # Get the base dataset directory
urls = [
    'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-train.zip',
    'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-val.zip',
    'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-dev.zip',
    'https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-challenge.zip'
]
download(urls, dir=dir, curl=True, threads=4)  # Download the datasets

# Convert annotations for training, validation, and test-dev sets
for d in 'VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev':
    visdrone2yolo(dir / d)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Converting dataset/VisDrone2019-DET-train: 6471it [00:03, 1827.97it/s]
Converting dataset/VisDrone2019-DET-val: 548it [00:00, 1725.38it/s]
Converting dataset/VisDrone2019-DET-test-dev: 1610it [00:00, 2003.21it/s]


In [3]:
# Save the YAML configuration
yaml_content = """
path: ./
train: VisDrone2019-DET-train/images
val: VisDrone2019-DET-val/images
test: VisDrone2019-DET-test-dev/images

# Number of classes
nc: 10

# Class names
names:
  0: pedestrian
  1: people
  2: bicycle
  3: car
  4: van
  5: truck
  6: tricycle
  7: awning-tricycle
  8: bus
  9: motor
"""
with open('VisDrone.yaml', 'w') as f:
    f.write(yaml_content)


In [4]:
import os
import shutil
from pathlib import Path

# Define the source folder and target folder
source_dir = Path('./dataset/')  # Adjust this to where your current dataset is
target_dir = Path('/kaggle/working/datasets/')

# Create the target directory if it doesn't exist
target_dir.mkdir(parents=True, exist_ok=True)

# List all subfolders in the source directory
subfolders = ['VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev']

# Move the subfolders to the target directory
for subfolder in subfolders:
    src_subfolder = source_dir / subfolder
    dest_subfolder = target_dir / subfolder
    if src_subfolder.exists():
        print(f"Moving {src_subfolder} to {dest_subfolder}")
        shutil.move(str(src_subfolder), str(dest_subfolder))
    else:
        print(f"Source folder {src_subfolder} not found, skipping.")

# After moving, verify the structure
print("Current structure under /kaggle/working/datasets/:")
print(os.listdir(target_dir))


Moving dataset/VisDrone2019-DET-train to /kaggle/working/datasets/VisDrone2019-DET-train
Moving dataset/VisDrone2019-DET-val to /kaggle/working/datasets/VisDrone2019-DET-val
Moving dataset/VisDrone2019-DET-test-dev to /kaggle/working/datasets/VisDrone2019-DET-test-dev
Current structure under /kaggle/working/datasets/:
['VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev']


In [5]:
from ultralytics import YOLO

# Charger un modèle pré-entraîné YOLOv8
model = YOLO('yolov8s.pt')  # Options: yolov8n.pt, yolov8m.pt, yolov8l.pt

# Entraînement du modèle
model.train(data='VisDrone.yaml', epochs=100, imgsz=640, batch=16)

100%|██████████| 21.5M/21.5M [00:00<00:00, 101MB/s] 


Ultralytics 8.3.37 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=VisDrone.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

100%|██████████| 755k/755k [00:00<00:00, 23.7MB/s]
2024-11-25 15:08:26,311	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-25 15:08:27,835	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 92.3MB/s]


...

                   all        548      38759      0.505      0.389      0.399      0.237



100 epochs completed in 3.878 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.37 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:09<00:00,  1.87it/s]


                   all        548      38759      0.515      0.391      0.405      0.243
            pedestrian        520       8844      0.523      0.407      0.433      0.197
                people        482       5125      0.584      0.282      0.335      0.129
               bicycle        364       1287      0.283      0.158      0.139     0.0599
                   car        515      14064       0.73      0.769      0.797      0.568
                   van        421       1975      0.516      0.463      0.463      0.324
                 truck        266        750      0.546      0.379       0.39       0.26
              tricycle        337       1045      0.408      0.325      0.293      0.167
       awning-tricycle        220        532      0.338      0.171      0.161      0.099
                   bus        131        251      0.681      0.514      0.586      0.424
                 motor        485       4886      0.542      0.438      0.452      0.201
Speed: 0.1ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a8d9afbc640>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0